In [1]:
import os
import itertools
import time
from typing import List, Tuple, Dict
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from tools.metrics import train_and_evaluate_model, cross_validate
from tools.knn import KNNClassifier
from tools.distance import (
    ManhattanDistance,
    EuclideanDistance,
    ChebyshevDistance,
    MahalanobisDistance,
)
from tools.voting import MajorityClassVote, InverseDistanceWeightedVote, ShepardsWorkVote
from tools.preprocess import (
    preprocess_hepatitis_datasets,
    load_datasets,
    preprocess_mushroom_datasets,
)
from tools.weighting import InformationGainWeighting, ReliefFWeighting, EqualWeighting
from tools.reduction import GCNN, ENNTH, drop3, edited_nearest_neighbor
import numpy as np
from sklearn.svm import SVC as SVMClassifier

import logging

import pathlib

In [2]:
DATA_DIR = "../data"
logging.basicConfig(level=logging.INFO)

In [3]:
def run_svm(
    train_dfs: List[pd.DataFrame],
    test_dfs: List[pd.DataFrame],
    dataset_name: str,
    class_columns_per_ds: Dict[str, str],
) -> None:
    """
    Run SVM classification with various parameter configurations.
    """
    # SVM Parameters
    c_values = [1, 3, 5, 7]
    kernel_types = ["linear", "poly", "rbf", "sigmoid"]

    # Results DF
    results_svm = pd.DataFrame(
        columns=[
            "C",
            "kernel_type",
            "fold1",
            "fold2",
            "fold3",
            "fold4",
            "fold5",
            "fold6",
            "fold7",
            "fold8",
            "fold9",
            "fold10",
        ]
    )

    # Run all parameter configurations
    for C, kernel_type in itertools.product(c_values, kernel_types):
        svm = SVMClassifier(
            C=C,
            kernel=kernel_type,
        )

        logging.debug(f"Running SVM: [kernel={kernel_type}, C={C}]")

        # Cross-validate
        f1_scores = cross_validate(svm, train_dfs, test_dfs, class_columns_per_ds[dataset_name], score_func=f1_score)

        # Append the results
        results_svm.loc[len(results_svm)] = [
            C,
            kernel_type,
            *f1_scores,
        ]

    # Save the results for SVM
    file_path_svm = os.path.join(DATA_DIR, "results_per_fold", f"svm_{dataset_name}.csv")
    results_svm.to_csv(file_path_svm, index=False)

In [4]:
def run_knn(
    train_dfs: List[pd.DataFrame],
    test_dfs: List[pd.DataFrame],
    dataset_name: str,
    class_columns_per_ds: Dict[str, str],
    full_data_X: pd.DataFrame,
    full_data_y: pd.Series,
) -> Tuple[KNNClassifier, Dict[str, np.ndarray]]:
    """
    Run KNN classification with various parameter configurations.
    """
    # Define CM for MahalanobisDistance
    # TODO: fix this for MahalanobisDistance
    # covariance_matrix = np.cov(train_dfs[0].drop(columns=[class_columns_per_ds[dataset_name]]).apply(pd.to_numeric, errors='coerce'),
    #                         rowvar=False)

    # KNN Parameters
    k_values = [1, 3, 5, 7]
    distance_funcs = [ManhattanDistance(), EuclideanDistance(), ChebyshevDistance()]
    voting_funcs = [MajorityClassVote(), InverseDistanceWeightedVote(), ShepardsWorkVote()]
    weighting_funcs = [EqualWeighting(), InformationGainWeighting(), ReliefFWeighting()]

    # Results DF
    results = pd.DataFrame(
        columns=[
            "k",
            "distance_func",
            "voting_func",
            "weighting_func",
            "fold1",
            "fold2",
            "fold3",
            "fold4",
            "fold5",
            "fold6",
            "fold7",
            "fold8",
            "fold9",
            "fold10",
        ]
    )

    logging.info("Fitting weighting functions")
    weights = {}
    for weighting_func in weighting_funcs:
        weighting_func.fit(full_data_X, full_data_y)
        weights[weighting_func.__class__.__name__] = weighting_func.get_weights()

    # Run all parameter configurations
    best_config_instance = None
    for k, distance_func, voting_func, weighting_func in itertools.product(
        k_values, distance_funcs, voting_funcs, weighting_funcs
    ):

        knn = KNNClassifier(
            k=k,
            distance_func=distance_func,
            voting_func=voting_func,
            weights=weights[weighting_func.__class__.__name__],
        )

        logging.debug(
            f"Running KNN: [weighting_func={weighting_func.__class__.__name__}, distance_func={distance_func.__class__.__name__}, voting_func={voting_func.__class__.__name__}, k={k}]"
        )

        f1_scores = cross_validate(
            knn, train_dfs, test_dfs, class_columns_per_ds[dataset_name], score_func=f1_score
        )

        # Append the results
        results.loc[len(results)] = [
            k,
            distance_func.__class__.__name__,
            voting_func.__class__.__name__,
            weighting_func.__class__.__name__,
            *f1_scores,
        ]

        # Store best configuration instance
        if best_config_instance is None or f1_scores.max() > results["fold1"].max():
            best_config_instance = {
                "k": k,
                "distance_func": distance_func,  # Save the instance
                "voting_func": voting_func,
                "weighting_func": weighting_func,
            }

    # Save the results for KNN
    file_path = os.path.join(DATA_DIR, "results_per_fold", f"knn_{dataset_name}.csv")
    results.to_csv(file_path, index=False)

    return best_config_instance, weights

In [5]:
def run_reduced_knn(
    train_dfs: List[pd.DataFrame],
    test_dfs: List[pd.DataFrame],
    dataset_name: str,
    class_columns_per_ds: Dict[str, str],
    best_config_instance: Dict[str, any],
    weights: Dict[str, np.ndarray],
) -> None:
    """
    Run reduced KNN classification with various techniques.
    """
    best_k = best_config_instance["k"]
    best_distance_func = best_config_instance["distance_func"]
    best_voting_func = best_config_instance["voting_func"]
    best_weighting_func = best_config_instance["weighting_func"]

    reduction_funcs = {
        "control": lambda x, y, z, s: (x, y),
        "GCNN": GCNN,
        "ENNTH": ENNTH,
        "Drop3": drop3,
    }
    reduction_results = pd.DataFrame(
        columns=[
            "k",
            "distance_func",
            "voting_func",
            "weighting_func",
            "reduction_func",
            "fold1",
            "fold2",
            "fold3",
            "fold4",
            "fold5",
            "fold6",
            "fold7",
            "fold8",
            "fold9",
            "fold10",
        ]
    )

    best_k = 7
    for reduction_func in reduction_funcs:
        knn = KNNClassifier(
            k=best_k,
            distance_func=best_distance_func,  # Use instance, not string
            voting_func=best_voting_func,
            weights=weights[best_weighting_func.__class__.__name__],  # We will apply weights later
        )

        logging.debug(f"Running KNN with reduction: {reduction_func}")

        f1_scores = []
        for train_df, test_df in zip(train_dfs, test_dfs):
            X_train = train_df.drop(columns=[class_columns_per_ds[dataset_name]])
            y_train = train_df[class_columns_per_ds[dataset_name]]
            X_test = test_df.drop(columns=[class_columns_per_ds[dataset_name]])
            y_test = test_df[class_columns_per_ds[dataset_name]]

            logging.debug(f"Reducing training data with {reduction_func}")
            X_train_reduced, y_train_reduced = reduction_funcs[reduction_func](
                X_train, y_train, best_k, knn
            )
            # Train and evaluate the KNN model
            y_true, y_pred, train_time, test_time = train_and_evaluate_model(
                knn, X_train_reduced, y_train_reduced, X_test, y_test
            )
            f1_scores.append(f1_score(y_true, y_pred))
            logging.debug(
                f"Reduced training data storage: {len(X_train_reduced)} / {len(X_train)}. Took {train_time} seconds to train and {test_time} seconds to test."
            )

        # Append the results
        reduction_results.loc[len(reduction_results)] = [
            best_k,
            best_distance_func.__class__.__name__,
            best_voting_func.__class__.__name__,
            best_weighting_func.__class__.__name__,
            reduction_func,
            *f1_scores,
        ]

    # Save the results for reduced KNN
    file_path_reduction = os.path.join(
        DATA_DIR, "results_per_fold", f"knn_reduction_{dataset_name}.csv"
    )
    reduction_results.to_csv(file_path_reduction, index=False)

In [6]:
# Configurations
dataset_name = "hepatitis"
sample = 1000

processing_funcs_per_ds = {
    "hepatitis": preprocess_hepatitis_datasets,
    "mushroom": preprocess_mushroom_datasets,
}
class_columns_per_ds = {"hepatitis": "Class", "mushroom": "class"}

logging.debug(f"Using {dataset_name} dataset...")
train_path = f"{DATA_DIR}/raw/{dataset_name}/*train.arff"
test_path = f"{DATA_DIR}/raw/{dataset_name}/*test.arff"
train_dfs = load_datasets(train_path)
test_dfs = load_datasets(test_path)


train_dfs = [df.head(sample) for df in train_dfs]
test_dfs = [df.head(sample) for df in test_dfs]

logging.debug(f"train_path: {train_path}")
train_dfs = [processing_funcs_per_ds[dataset_name](df) for df in train_dfs]
test_dfs = [processing_funcs_per_ds[dataset_name](df) for df in test_dfs]
logging.debug(f"Train datasets count: {len(train_dfs)}")
logging.debug(f"Test datasets count: {len(test_dfs)}")

full_data = pd.concat([train_dfs[0], test_dfs[0]])
full_data_X = full_data.drop(columns=[class_columns_per_ds[dataset_name]])
full_data_y = full_data[class_columns_per_ds[dataset_name]]

start_time = time.time()

# ========== SVM ==========

# logging.info("Running SVM...")
# run_svm(train_dfs, test_dfs, dataset_name, class_columns_per_ds)

# ========== KNN ==========

logging.info("Running KNN...")
best_config_instance, weights = run_knn(
    train_dfs, test_dfs, dataset_name, class_columns_per_ds, full_data_X, full_data_y
)

# ========== Reduced KNN ==========

logging.info("Running reduced KNN...")
run_reduced_knn(
    train_dfs, test_dfs, dataset_name, class_columns_per_ds, best_config_instance, weights
)

logging.info(f"Finished in {time.time() - start_time} seconds")

INFO:root:Running KNN...
INFO:root:Fitting weighting functions
INFO:root:Running reduced KNN...
INFO:root:Finished in 16.743202209472656 seconds
